In [1]:
#@title ***Getting drive content***

from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [2]:
#@title ***Importing required librarys***

import math
import numpy as np
import cv2 as cv
import glob
from tensorflow import keras
from sklearn import svm
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score
import joblib   

In [3]:
#@title ***Project path***

project_path = "/content/drive/MyDrive/Colab Notebooks/Ml_Project"

In [4]:
#@title ***Reading images utility***

def read_images(gray_scale=False):
  print('Reading images...')
  images = []
  labels = []

  for label in range(0, 10):
    path = f"{project_path}/Dataset/{label}/*.JPG"
      
    for img_path in glob.glob(path):
      # reading image
      img = cv.imread(img_path)
      # resize image 
      img = cv.resize(img, (100, 100))
      
      if gray_scale:
        # convert image from BGR to gary
        img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
      else:
        # convert image from BGR to RGB
        img = cv.cvtColor(img, cv.COLOR_BGR2RGB)       
        
      images.append(img)
      labels.append([1 if i == label else 0 for i in range(0, 10)])

  images = np.array(images)
  labels = np.array(labels)
    
    
  # normalize images
  if not gray_scale:
    # calculate average
    avg = np.average(images, axis=0)
    # subtract averages from each image.
    images = images - avg
      
  # divide each image by 255
  images = images / 255.0 

  print('Images read successfully.\n')

  return (images, labels)

In [5]:
#@title ***Calculating fscore utility***

def calc_fscore(recall, precision):
  return 2 * ((recall * precision) / (recall + precision))

In [6]:
#@title ***Splitting data utility***

def split_data(features, output, train_size): 
  return train_test_split(features, output, train_size=train_size)

In [7]:
#@title ***Writing model report utility***

def write_model_report(file_name, measurements={}):
  with open(f"{project_path}/{file_name}.txt", "a") as file: 
    for m in measurements:    
      file.write(f"{m} = {measurements[m]}\n")

In [8]:
#@title ***Running NN model utility***

def run_nn_model(model_id, model: keras.models.Sequential, images, labels, train_size=0.8, epochs=30):
  # Showing model summary
  with open(f"{project_path}/{model_id}.txt", "w") as file:
    model.summary(print_fn=lambda x: file.write(x + '\n'))
    file.write('\n')
  
  # Spliting data into training & testing
  training_images, testing_images, training_labels, testing_labels = split_data(images, labels, train_size)

  # Configuring model
  model.compile(
    optimizer='adam', 
    loss='categorical_crossentropy', 
    metrics=['accuracy', keras.metrics.Recall(), keras.metrics.Precision()]
  )
  
  # Fitting model
  model.fit(
    training_images, 
    training_labels, 
    epochs=epochs, 
    verbose=1,
    validation_data=(testing_images, testing_labels),
  )
  
  # Evaluating model
  loss, accuracy, recall, precision = model.evaluate(testing_images, testing_labels, verbose=2)
  
  # Saving model
  model.save(f"{project_path}/{model_id}.h5")
  
  # Reporting model 
  write_model_report(model_id, {
    "loss": loss,
    "accuracy": accuracy,
    "recall": recall,
    "precision": precision,
    "fscore": calc_fscore(recall, precision)
  })

In [ ]:
#@title ***Model 1***

# Reading images & labels
images, labels = read_images(True)

# Setting model up
model = keras.models.Sequential([
  keras.layers.Flatten(input_shape=(100, 100)),
  keras.layers.Dense(1024, activation='relu'),
  keras.layers.Dense(512, activation='relu'),
  keras.layers.Dense(10, activation='softmax')
])

run_nn_model('model1', model, images, labels)

In [ ]:
#@title ***Model 2***

# Reading images & labels
images, labels = read_images(True)

# Setting model up
model = keras.models.Sequential([
  keras.layers.Flatten(input_shape=(100, 100)),
  keras.layers.Dense(10000, activation='relu'),
  keras.layers.Dense(10, activation='softmax')
])

run_nn_model('model2', model, images, labels)

In [ ]:
#@title ***Model 3***

# Reading images & labels
images, labels = read_images()

# Setting model up
model = keras.models.Sequential([
  keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(100, 100, 3)),
  keras.layers.MaxPool2D((2, 2)),
  keras.layers.Conv2D(64, (3, 3), activation='relu'),
  keras.layers.MaxPool2D((2, 2)),
  keras.layers.Flatten(),
  keras.layers.Dense(1024, activation='relu'),
  keras.layers.Dense(512, activation='relu'),
  keras.layers.Dense(10, activation='softmax')
])

run_nn_model('model3', model, images, labels)

In [ ]:
#@title ***Model 4***

# Reading images & labels
images, labels = read_images()

# Flatting images
images = np.array([img.flatten() for img in images])

# reformat labels
labels = np.argmax(labels, axis=1)


# Spliting data into training & testing
training_images, testing_images, training_labels, testing_labels = split_data(images, labels, 0.8)

# model classifier
model = svm.SVC(C=10, gamma=0.001)

# training model
print('Model fitting has been started\n')
model.fit(training_images, training_labels)
print('Model fitting has been finished\n')

# saving model 
joblib.dump(model, f"{project_path}/model4.pkl")

# making prediction
predicted_labels = model.predict(testing_images)

# reporting model
with open(f"{project_path}/model4.txt", "w") as file: 
      file.write(f"{model}\n\n")

recall = recall_score(testing_labels, predicted_labels, average='micro')
precision = precision_score(testing_labels, predicted_labels, average='micro')

write_model_report("model4", {
  "accuracy": accuracy_score(testing_labels, predicted_labels),
  "recall": recall,
  "precision": precision,
  "fscore": calc_fscore(recall, precision)
})